In [ ]:
%cd /content

# download the predictive coding repository
!git clone https://github.com/jgornet/predictive-coding-recovers-maps.git
%cd predictive-coding-recovers-maps/notebooks
!pip install -r ../requirements.txt

In [ ]:
%pushd ../weights
!python download.py
%popd

In [ ]:
from pathlib import Path
import os

import matplotlib
import matplotlib.pyplot as plt

import numpy as np

import torch
import torch.nn.functional as F
import torch.optim

from predictive_coding.dataset import collate_fn, EnvironmentDataset
from predictive_coding.trainer import Trainer
from predictive_coding import Autoencoder, PredictiveCoder

# Training a predictive coding neural network

In this Google Colab notebook, we detail the procedure for training a predictive coding neural network using a dataset derived from a Minecraft environment.

## Loading the Dataset

The dataset captures sequences of an agent's movements within the Minecraft environment.
Before using this data, certain preprocessing steps might be necessary, such as normalization or reshaping, depending on the nature and format of the data. It's crucial to divide the dataset into two distinct sets. The training set is used to adjust the model's weights, while the validation set helps evaluate the model's performance on unseen data and prevent overfitting.


In [ ]:
train_dataset = EnvironmentDataset(Path("../datasets/train-dataset"))
train_dataloader = torch.utils.data.DataLoader(
    train_dataset,
    batch_size=64,
    shuffle=True,
    collate_fn=collate_fn,
    num_workers=4,
    pin_memory=True,
)

val_dataset = EnvironmentDataset(Path("../datasets/val-dataset"))
val_dataloader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=100,
    shuffle=False,
    collate_fn=collate_fn,
    num_workers=4,
    pin_memory=True,
)


# Neural Network and Optimizer Setup

The predictive coding neural network is designed to forecast future states based on the current and past states. This makes it suitable for understanding sequences like our Minecraft data. An optimizer aids in updating the model's weights. Common choices include Adam, SGD, and RMSprop. The optimizer's role is to minimize the error between the predicted and actual outcomes, adjusting the model's weights in the process.


In [ ]:
experiment_name = 'predictive-coding'
model = PredictiveCoder(in_channels=3, out_channels=3, layers=[2, 2, 2, 2], seq_len=20)
model = model.to('cuda:0')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-1, momentum=0.9, weight_decay=5e-6)
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=1e-1, epochs=200, steps_per_epoch=len(train_dataloader))


## Neural Network Training

Rather than feeding the entire dataset at once, it's broken into smaller chunks or batches. This makes the training process more manageable and often leads to better convergence.
After each batch is processed, the model's prediction is compared to the actual outcome. Based on this comparison, the optimizer adjusts the neural network's weights to reduce prediction error.
Iterations: This batch processing and weight adjustment are repeated multiple times (epochs) until the model's performance plateaus or meets a predetermined criterion.
By following these steps meticulously, you can efficiently train your predictive coding neural network on the Minecraft dataset.


In [ ]:
ckpt_path = os.path.abspath('./experiments/' + experiment_name)
if not os.path.exists(ckpt_path):
    os.makedirs(ckpt_path, exist_ok=True)
trainer = Trainer(model, optimizer, scheduler, train_dataloader, val_dataloader,
                  checkpoint_path=ckpt_path)
trainer.fit(num_epochs=200)


### The circular environment

In this section of our Google Colab notebook, we shift our focus to the implementation of a predictive coder neural network within the circular corridor environment in Minecraft. Unlike the autoencoder, the predictive coder's primary objective is to forecast future images based on a sequence of past observations. Leveraging the temporal dependencies inherent in the environment, the network learns to anticipate the next frame in the sequence, effectively predicting how the scene will evolve over time. Similar to the autoencoder, we employ gradient descent to optimize the predictive coder's parameters, allowing it to progressively refine its predictions and generate increasingly accurate forecasts of future images.


In [ ]:
dataset = CircleDataset(
    '../datasets/circle-dataset/circle_images.npy', 
    '../datasets/circle-dataset/circle_positions.npy', 
    length=30, 
    speed=5
)
train_dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=64,
    shuffle=True,
    collate_fn=collate_fn,
    num_workers=2,
    pin_memory=True,
)

val_dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=64,
    shuffle=False,
    collate_fn=collate_fn,
    num_workers=2,
    pin_memory=True,
)


In [ ]:
experiment_name = 'predictive-coding-circle'
model = PredictiveCoder(in_channels=3, out_channels=3, layers=[2, 2, 2, 2], seq_len=30)
model = model.to('cuda:0')
optimizer = torch.optim.SGD(model.parameters(), lr=5e-2, momentum=0.9, weight_decay=5e-6)
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=5e-2, epochs=400, steps_per_epoch=len(train_dataloader))


# Training an autoencoder

We focus on training an autoencoder using a dataset derived from a Minecraft environment. The dataset captures attributes from an agent's movements within Minecraft and may require preprocessing, such as normalization and reshaping, to ensure compatibility with the autoencoder structure. After dividing the data into training and validation segments, the autoencoder's architecture—comprising an encoder that compresses the input and a decoder that reconstructs it—is set up. The aim is to minimize the difference between the original data and its reconstruction. An optimizer, like Adam or SGD, is employed to refine the model's weights based on the observed reconstruction error. Throughout the training phase, the model processes data in batches, using feedback from each batch to adjust its weights and improve the fidelity of the data reconstruction. This iterative process continues until the model achieves satisfactory performance or until the improvement plateau is reached.


In [ ]:
experiment_name = 'autoencoder'
model = Autoencoder(in_channels=3, out_channels=3, layers=[2, 2, 2, 2])
model = model.to('cuda:0')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-1, momentum=0.9, weight_decay=5e-6)
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=1e-1, epochs=200, steps_per_epoch=len(train_dataloader))

ckpt_path = os.path.abspath('./experiments/' + experiment_name)
if not os.path.exists(ckpt_path):
    os.makedirs(ckpt_path, exist_ok=True)
trainer = Trainer(model, optimizer, scheduler, train_dataloader, val_dataloader,
                  checkpoint_path=ckpt_path)
trainer.fit(num_epochs=200)


### The circular environment

In this section of our Google Colab notebook, we delve into the practical implementation of training an autoencoder neural network within the circular corridor environment in Minecraft. Mirroring our approach in the prior environment, the autoencoder's primary objective is to minimize the discrepancy between the original input image and its reconstructed counterpart. To achieve this, we employ the standard gradient descent algorithm, iteratively updating the autoencoder's parameters using the available training data. This process allows the network to progressively learn the most salient features of the environment, ultimately resulting in a model capable of generating accurate reconstructions.


In [ ]:
dataset = CircleDataset(
    '../datasets/circle-dataset/circle_images.npy', 
    '../datasets/circle-dataset/circle_positions.npy', 
    length=30, speed=5
)
train_dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=64,
    shuffle=True,
    collate_fn=collate_fn,
    num_workers=2,
    pin_memory=True,
)

val_dataloader = torch.utils.data.DataLoader(
    dataset,
    batch_size=64,
    shuffle=False,
    collate_fn=collate_fn,
    num_workers=2,
    pin_memory=True,
)


In [ ]:
experiment_name = 'autoencoder-circle'
model = Autoencoder(in_channels=3, out_channels=3, layers=[2, 2, 2, 2])
model = model.to('cuda:0')
optimizer = torch.optim.SGD(model.parameters(), lr=1e-2, momentum=0.9, weight_decay=5e-6)
scheduler = torch.optim.lr_scheduler.OneCycleLR(optimizer, max_lr=1e-2, epochs=200, steps_per_epoch=len(train_dataloader))

ckpt_path = os.path.abspath('./experiments/' + experiment_name)
if not os.path.exists(ckpt_path):
    os.makedirs(ckpt_path, exist_ok=True)
trainer = Trainer(model, optimizer, scheduler, train_dataloader, val_dataloader,
                  checkpoint_path=ckpt_path)
trainer.fit(num_epochs=200)
